In [1]:
import sys
sys.path.append('../')
import os

In [2]:
from application.review_obj_class import ReviewObj
from application.namespan_class import NameSpanGenerator
import pandas as pd
%pprint

Pretty printing has been turned OFF


In [184]:
directory = "../../aps_reviews_1000/"
filenames = os.listdir(directory)
reviews = ((file.split('.')[0], open(directory + file).read()) for file in filenames)

In [185]:
review_list = [ReviewObj(file, txt) for (file, txt) in reviews]

KeyboardInterrupt: 

In [5]:
ex = NameSpanGenerator.generate(review_list[20])

In [6]:
[x.collocates for x in ex.spans]

[['tribute', 'to', 'Dr._Gladden', "'s", 'zealous'], ['writers', 'of', 'Dr._Gladden', "'s", 'school'], ['probably', 'what', 'Dr._Gladden', 'means', ','], ['country', '.', 'Dr._Gladden', 'is', 'not']]

In [7]:
list(set((x for x in review_list[0].cleaned_text[:50].split() if x.isalnum())))

['Rl', 'New']

In [8]:
[x.name for x in ex.spans]

[Dr. Gladden, Dr. Gladden, Dr. Gladden, Dr. Gladden]

In [9]:
for x in review_list:
    x = NameSpanGenerator.generate(x)

In [10]:
example_reviews = [x for x in review_list if (len(x.person_names) > 3) and (len(x.pub_names) > 1)]

In [11]:
len(example_reviews)

77

In [12]:
[x.name for x in example_reviews[0].spans]

[Dr. Leopold Damrosch, Mr. Andrew Carnegie, Dr. LeopOld Damrosch, Prof. Horatio W. Parker, Mr. Parker, Lord. The, Oratorio Society, Oratorio Society, Mendelssohn Society, Symphony Society]

In [13]:
name_indices = [x.name.review_loc_toks for x in example_reviews[0].spans]

In [14]:
name_indices

[244, 423, 459, 493, 561, 731, 3, 64, 284, 433]

In [15]:
example_reviews[0].coll_toks_ind[name_indices[0]-2:name_indices[0]+3]

['z873', 'that', 'Dr._Leopold_Damrosch', 'united', 'a']

## Looking through the collocations

In [17]:
review_list[20].spans

[<application.namespan_class.NameSpan object at 0x1a3beb2610>, <application.namespan_class.NameSpan object at 0x1a3beb2850>, <application.namespan_class.NameSpan object at 0x1a3beb2910>, <application.namespan_class.NameSpan object at 0x1a3beb2950>]

In [26]:
[span for spans in (rev.spans for rev in review_list[:2]) for span in spans]

[<application.namespan_class.NameSpan object at 0x1a3beb0a10>, <application.namespan_class.NameSpan object at 0x1a2115e310>, <application.namespan_class.NameSpan object at 0x1a3beb0ad0>, <application.namespan_class.NameSpan object at 0x1a3beb0a90>, <application.namespan_class.NameSpan object at 0x1a3beb0b10>, <application.namespan_class.NameSpan object at 0x1a3beb0c50>, <application.namespan_class.NameSpan object at 0x1a3beb09d0>, <application.namespan_class.NameSpan object at 0x1a3beb0b90>, <application.namespan_class.NameSpan object at 0x1a3beb0bd0>, <application.namespan_class.NameSpan object at 0x1a3beb0c10>, <application.namespan_class.NameSpan object at 0x1a3beb0c90>, <application.namespan_class.NameSpan object at 0x1a3beb0b50>, <application.namespan_class.NameSpan object at 0x1a3beb0d10>, <application.namespan_class.NameSpan object at 0x1a3beb0d50>, <application.namespan_class.NameSpan object at 0x1a3beb0d90>, <application.namespan_class.NameSpan object at 0x1a3beb0dd0>, <applic

In [27]:
spans = [span for spans in (rev.spans for rev in review_list) for span in spans]

In [28]:
len(spans)

5839

In [29]:
person_spans = [span for span in spans if span.label == 'person']

In [30]:
len(person_spans)

5192

In [72]:
coll_df = pd.DataFrame()

In [73]:
coll_df['review_id'] = list(map((lambda x: x.review_id), person_spans))
coll_df['review_group'] = list(map((lambda x: x.group), person_spans))

In [74]:
person_spans[0].collocates

['Place', 'and', 'Mrs._Canfield', "'s", 'The']

In [76]:
set([len(span.collocates) for span in person_spans])

{0, 3, 4, 5}

### Problem situations:
    
+ Okay: 0 1 name(2) 3 4
    + 2 !< 2
+ Bad: X 0 name(1) 2 3
    + 1 < 2
    + **2 - 1 = 1**
+ Bad: X X name(0) 2 3
    + 0 < 2
    + **2 - 0 = 2**

**Pattern: span.name.review_loc_toks < 2**

+ Okay: -5 -4 name(-3) -2 -1
    + 0 1 name(2) 3 4
    + 5 toks, 5 - 3 = 2
    + 2 !> 2
+ Bad: -4 -3 name(-2) -1 X
    + 0 1 name(2) 3
    + 4 toks, 4 - 3 = 1
    + 2 > 1
    + **2 - 1 = 1**
+ Bad: -3 -2 name(-1) X X
    + 0 1 name(2)
    + 3 toks, 3 - 3 = 0
    + 2 > 0
    + **2 - 0 = 2**
+ Bad: -4 -3 name(-2) -1 X
    + 300 301 302(name) 303 X
    + 304 toks, 304 - 3 = 301
    + 302 > 301
    + **302 - 301 = 1**
+ Bad: -3 -2 name(-1) X X
    + 302 303(name) X X
    + 304 toks, 304 - 3 = 301
    + 303 > 301
    + **303 - 301 = 2**

**Pattern: span.name.review_loc_toks > (len(toks)-3)**

In [113]:
len([span.collocates for span in person_spans if len(span.collocates) < 5])

26

In [114]:
[span.collocates for span in person_spans if len(span.collocates) < 5]

[['literature', '.', 'Prof._H._P._Forbes', '.'], [], [], [], [], ['Englishmen', 'as', 'Lord_Ronald_Gower', ','], [], [], [], ['Next', 'exclaims', 'Lord_Tennyson_Mary_Gay_Humphreys'], [], [], [], [',', 'by', 'Dr._Rubens_Hirschberg', '.'], ['etching', 'by', 'Mr._Herbert_Railton', '.'], [], ['Is', 'by', 'Mr._Farjeon', '.'], ['disciple', 'of', 'Mr._Allen', '.'], [], [], ['treatment', 'of', 'Sir_Robert_Peel', '.'], [], [], ['bishop', 'than', 'Dr._Sattcrlec', '.'], [], []]

In [172]:
problem_ids = [span.review_id for span in person_spans if len(span.collocates) < 5]

In [178]:
problem_revs = [rev for rev in review_list if rev.review_id in problem_ids]

In [179]:
len(problem_revs)

26

In [181]:
for review in problem_revs:
    for span in review.spans:
        toks = review.coll_toks_ind
        ni = span.name.review_loc_toks
#         if (ni < 2):
#             print(ni, "front")
#             ftoks = list((2-ni)*"^")
#             ftoks.extend((review.coll_toks_ind))
#             print(ftoks[0:(5-ni)])
        if (ni >= (len(toks)-3)):
            print("ni:", ni)
            print("toks - 3:", len(toks), "- 3 =", len(toks)-3)
            print(review.coll_toks_ind[ni:])
#             print(ni, "back")
#             btoks = review.coll_toks_ind
#             btoks.extend(list((ni-(len(review.coll_toks_ind)-3)) * "$"))
#             print(btoks[-10:])

ni: 3878
toks - 3: 3881 - 3 = 3878
['Prof._H._P._Forbes', '.', '$']
ni: 480
toks - 3: 483 - 3 = 480
['Lord_Ronald_Gower', ',', '$']
ni: 1459
toks - 3: 1462 - 3 = 1459
['Lord_Tennyson_Mary_Gay_Humphreys', '$', '$']
ni: 356
toks - 3: 359 - 3 = 356
['Dr._Rubens_Hirschberg', '.', '$']
ni: 606
toks - 3: 609 - 3 = 606
['Mr._Herbert_Railton', '.', '$']
ni: 532
toks - 3: 535 - 3 = 532
['Mr._Farjeon', '.', '$']
ni: 977
toks - 3: 980 - 3 = 977
['Mr._Allen', '.', '$']
ni: 1742
toks - 3: 1745 - 3 = 1742
['Sir_Robert_Peel', '.', '$']
ni: 740
toks - 3: 743 - 3 = 740
['Dr._Sattcrlec', '.', '$']


In [75]:
coll_df['preceding_collocates'] = list(map((lambda x: x.collocates[:2]), person_spans))
coll_df['following_collocates'] = list(map((lambda x: x.collocates[3:]), person_spans))

coll_df['coll-2'] = list(map((lambda x: x.collocates[0]), person_spans))
coll_df['coll-1'] = list(map((lambda x: x.collocates[1]), person_spans))
coll_df['name_target'] = list(map((lambda x: x.name), person_spans))
coll_df['coll+1'] = list(map((lambda x: x.collocates[3]), person_spans))
coll_df['coll+2'] = list(map((lambda x: x.collocates[4]), person_spans))

IndexError: list index out of range

In [66]:
coll_df.head()

,review_id,review_group,preceding_collocates,name_target,following_collocates
0,137197814,21,"[Place, and]",Mrs. Canfield,"['s, The]"
1,137197814,22,"[it, is]",Mr. Anderson,"['s, own]"
2,137197814,1,"[Years, ago]",Mr. George Moore,"[,, writing]"
3,137197814,26,"[Matisse, .]",Mr. Swinnerton,"[is, a]"
4,137197814,33,"[house, .]",Mr. Hergesheimer,"[seems, to]"
